In [1]:
import tensorflow as tf

In [2]:
model =tf.keras.models.load_model(filepath='facenet_keras.h5',compile=False)

In [3]:
print(model.inputs)
print(model.outputs)

[<tf.Tensor 'input_1:0' shape=(None, 160, 160, 3) dtype=float32>]
[<tf.Tensor 'Bottleneck_BatchNorm/Identity:0' shape=(None, 128) dtype=float32>]


In [4]:
import os
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray

In [5]:
import cv2
import matplotlib.pyplot as plt
import dlib
from imutils import face_utils
font = cv2.FONT_HERSHEY_SIMPLEX

In [6]:
import numpy as np

In [7]:
def extract_face(path, required_size=(160, 160)): 
    gray = cv2.imread(path, 0) 
    pixels = asarray(gray) 
    face_detect = dlib.get_frontal_face_detector() 
    rects = face_detect(gray, 1) 
    im = np.float32(gray) / 255.0 
    gx = cv2.Sobel(im, cv2.CV_32F, 1, 0, ksize=1) 
    gy = cv2.Sobel(im, cv2.CV_32F, 0, 1, ksize=1) 
    mag, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True) 
    for (i, rect) in enumerate(rects): 
        (x1, y1, w, h) = face_utils.rect_to_bb(rect) 
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + w, y1 + h
        face = pixels[y1:y2,x1:x2]
        image = Image.fromarray(face)
        image = image.resize(required_size)
        image = image.convert('RGB')
        face = asarray(image)
    return face

In [8]:
def load_faces(directory):
    faces = list()
    for filename in listdir(directory):
        path = directory + filename
        # get face
        face = extract_face(path)
        # store
        faces.append(face)
    return faces

In [9]:
# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
    X, y = list(), list()
    for subdir in listdir(directory):
        path = directory + subdir + '/'
        if not os.path.isdir(path):
            continue
        faces = load_faces(path)
        labels = [subdir for _ in range(len(faces))]
        print('>loaded %d examples for class: %s' % (len(faces), subdir))
        X.extend(faces)
        y.extend(labels)
    return asarray(X), asarray(y)

In [10]:
trainX, trainy = load_dataset('faces-dataset/train/')
print(trainX.shape, trainy.shape)
testX, testy = load_dataset('faces-dataset/val/')
print(testX.shape, testy.shape)
np.savez_compressed('faces-dataset2.npz', trainX, trainy, testX, testy)

>loaded 20 examples for class: Afreen
>loaded 20 examples for class: Karthik
>loaded 20 examples for class: Pranathi
>loaded 20 examples for class: Roshan
>loaded 20 examples for class: Samhitha
>loaded 20 examples for class: Shashi
>loaded 20 examples for class: Soumya
>loaded 20 examples for class: Srija
>loaded 20 examples for class: Srilatha
>loaded 20 examples for class: Suchitra
>loaded 20 examples for class: Suraj
>loaded 20 examples for class: Tejaswini
>loaded 20 examples for class: Yashaswi
(260, 160, 160, 3) (260,)
>loaded 6 examples for class: Afreen
>loaded 6 examples for class: Karthik
>loaded 6 examples for class: Pranathi
>loaded 6 examples for class: Roshan
>loaded 6 examples for class: Samhitha
>loaded 6 examples for class: Shashi
>loaded 6 examples for class: Soumya
>loaded 6 examples for class: Srija
>loaded 6 examples for class: Srilatha
>loaded 6 examples for class: Suchitra
>loaded 6 examples for class: Suraj
>loaded 6 examples for class: Tejaswini
>loaded 6 exam

In [12]:
# calculate a face embedding for each face in the dataset using facenet
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model

# get the face embedding for one face
def get_embedding(model, face_pixels):
    face_pixels = face_pixels.astype('float32')
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    samples = expand_dims(face_pixels, axis=0)
    yhat = model.predict(samples)
    return yhat[0]


data = np.load('faces-dataset2.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)
model = tf.keras.models.load_model('facenet_keras.h5')
print('Loaded Model')
# convert each face in the train set to an embedding
newTrainX = list()
for face_pixels in trainX:
	embedding = get_embedding(model, face_pixels)
	newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)
# convert each face in the test set to an embedding
newTestX = list()
for face_pixels in testX:
	embedding = get_embedding(model, face_pixels)
	newTestX.append(embedding)
newTestX = asarray(newTestX)
print(newTestX.shape)
# save arrays to one file in compressed format
savez_compressed('faces-embeddings2.npz', newTrainX, trainy, newTestX, testy)

Using TensorFlow backend.


Loaded:  (260, 160, 160, 3) (260,) (78, 160, 160, 3) (78,)
Loaded Model
(260, 128)
(78, 128)


In [13]:
from numpy import load
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
data = np.load('faces-embeddings2.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Dataset: train=%d, test=%d' % (trainX.shape[0], testX.shape[0]))
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)
model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainy)
yhat_train = model.predict(trainX)
yhat_test = model.predict(testX)
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

Dataset: train=260, test=78
Accuracy: train=100.000, test=100.000
